In [33]:

#################### imports ####################

import time
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

#################### Read csvs ####################
train_data = pd.read_csv('train_ml.csv', index_col=0)
test_data = pd.read_csv('test_ml.csv', index_col=0)

### preview ###
train_data.head()
#test_data.head()

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,updates,personal,promotions,forums,purchases,travel,spam,social
0,"Mon, 15 Oct 2018 08:03:09 +0000 (UTC)",researchgatemail,net,0,0,multipart/alternative,4,28,0,1,47.0,25556,0,1,0,0,0,0,0,1
1,"Thu, 17 Apr 2014 09:12:33 -0700 (PDT)",no-ip,com,0,0,multipart/alternative,6,32,0,0,46.0,19930,1,1,0,0,0,0,0,0
2,"Thu, 27 Oct 2016 01:36:28 +0000",mail,goodreads.com,0,0,multipart/mixed,0,0,0,0,21.0,4,0,1,0,0,0,0,0,1
3,"Fri, 16 Oct 2020 08:06:31 +0000 (GMT)",insideapple,apple.com,0,0,multipart/alternative,108,171,0,0,52.0,96568,0,0,1,0,0,0,0,0
4,"Thu, 02 Jul 2015 10:16:46 +0000",twitter,com,0,0,multipart/alternative,20,166,0,0,81.0,95131,0,1,0,0,0,0,0,1


In [34]:
### adding mailtype
mt1,mt2=[],[]
for mt in train_data['mail_type']:
    if isinstance(mt,str):
        mt1.append(mt.split('/')[0])
        if len(mt.split('/'))>1:
            mt2.append(mt.split('/')[1])
        else:
            mt2.append('None')
    else:
        mt1.append('None')
        mt2.append('None')

train_data['mt1']=mt1
train_data['mt2']=mt2

mt1,mt2=[],[]
for mt in test_data['mail_type']:
    if isinstance(mt,str):
        mt1.append(mt.split('/')[0])
        if len(mt.split('/'))>1:
            mt2.append(mt.split('/')[1])
        else:
            mt2.append('None')
    else:
        mt1.append('None')
        mt2.append('None')

test_data['mt1']=mt1
test_data['mt2']=mt2

In [36]:




### adding hour and date
hh,mm,ss,d,m,y,fus,fus2=[],[],[],[],[],[],[],[]
for date in train_data['date']:
    hh.append(int(date.split(':')[0].split(' ')[-1]))
    
    mm.append(int(date.split(':')[1]))
    
    ss.append(int(date.split(':')[2].split(' ')[0]))
    
    if len(date.split(':')[0].split(',')[0])==3:
        d.append(str(date.split(':')[0].split(',')[0]))
    else:
        d.append('None')
    
    if len(date.split(':')[0].split(' '))==5:
        if len(date.split(':')[0].split(' ')[2])==3:
            m.append(str(date.split(':')[0].split(' ')[2]))
        else:
            m.append('None')
    else:
        m.append(0)
    
    if len(date.split(':')[0].split(' '))==5:
        if len(date.split(':')[0].split(' ')[3])==4:
            y.append(str(date.split(':')[0].split(' ')[3]))
        else:
            y.append('None')
    else:
        y.append('None')
    
    if len(date.split())>=6:
        fus.append(date.split()[5])
        if len(date.split())>=7:
            fus2.append(date.split()[6])
        else:
            fus2.append('None')
    else:
        fus.append('None')
        fus2.append('None')

train_data['hh']=hh
train_data['mm']=mm
train_data['ss']=ss
train_data['d']=d
train_data['m']=m
train_data['y']=y
train_data['fus']=fus
train_data['fus2']=fus2

hm=[]
for i in range(len(hh)):
    hm.append(float(hh[i])+float(mm[i])/60)
train_data['hm']=hm

s0=[]
for s in ss:
    if ss=='00':
        s0.append(1)
    else:
        s0.append(0)
train_data['s0']=s0

weekend=['Sat','Sun']

w=[]
for day in d:
    if day in weekend:
        w.append(0)
    else:
        w.append(1)

train_data['w']=w

hh,mm,ss,d,m,y,fus,fus2=[],[],[],[],[],[],[],[]
for date in test_data['date']:
    hh.append(int(date.split(':')[0].split(' ')[-1]))
    
    mm.append(int(date.split(':')[1]))
    
    ss.append(int(date.split(':')[2].split(' ')[0]))
    
    if len(date.split(':')[0].split(',')[0])==3:
        d.append(str(date.split(':')[0].split(',')[0]))
    else:
        d.append('None')
    
    if len(date.split(':')[0].split(' '))==5:
        if len(date.split(':')[0].split(' ')[2])==3:
            m.append(str(date.split(':')[0].split(' ')[2]))
        else:
            m.append('None')
    else:
        m.append(0)
    
    if len(date.split(':')[0].split(' '))==5:
        if len(date.split(':')[0].split(' ')[3])==4:
            y.append(str(date.split(':')[0].split(' ')[3]))
        else:
            y.append('None')
    else:
        y.append('None')
    
    if len(date.split())>=6:
        fus.append(date.split()[5])
        if len(date.split())>=7:
            fus2.append(date.split()[6])
        else:
            fus2.append('None')
    else:
        fus.append('None')
        fus2.append('None')

test_data['hh']=hh
test_data['mm']=mm
test_data['ss']=ss
test_data['d']=d
test_data['m']=m
test_data['y']=y
test_data['fus']=fus
test_data['fus2']=fus2

hm=[]
for i in range(len(hh)):
    hm.append(float(hh[i])+float(mm[i])/60)
test_data['hm']=hm

s0=[]
for s in ss:
    if ss=='00':
        s0.append(1)
    else:
        s0.append(0)
test_data['s0']=s0

weekend=['Sat','Sun']

w=[]
for day in d:
    if day in weekend:
        w.append(0)
    else:
        w.append(1)

test_data['w']=w

test_data.head()

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,...,mm,ss,d,m,y,fus,fus2,hm,s0,w
0,"Thu, 13 Jul 2017 08:55:57 +0000",twitter,com,0,0,multipart/alternative,7,56,0,0,...,55,57,Thu,Jul,2017,+0000,None,8.916667,0,1
1,"Sun, 30 Sep 2018 14:42:12 +0000",mailer,netflix.com,0,0,multipart/alternative,5,33,0,0,...,42,12,Sun,Sep,2018,+0000,None,14.700000,0,0
2,"Mon, 13 Feb 2017 10:47:00 +0530",iiitd,ac.in,0,0,text/plain,0,2,1,0,...,47,0,Mon,Feb,2017,+0530,None,10.783333,0,1
3,"Thu, 16 Jun 2016 09:56:23 +0000",twitter,com,0,0,multipart/alternative,8,53,0,0,...,56,23,Thu,Jun,2016,+0000,None,9.933333,0,1
4,"Mon, 18 Apr 2016 01:51:59 +0530",iiitd,ac.in,0,0,multipart/mixed,0,0,0,0,...,51,59,Mon,Apr,2016,+0530,None,1.850000,0,1


In [37]:
#################### Filtering columns ####################
### numeric features
train_x_num = train_data.iloc[:,[3,4,6,7,8,9,10,11,-3,-2,-1]]
train_x_num = train_x_num.fillna(value=0)

train_y = train_data[['updates', 'personal', 'promotions',
                        'forums', 'purchases', 'travel',
                        'spam', 'social']]

test_x_num = test_data.iloc[:,[3,4,6,7,8,9,10,11,-3,-2,-1]]
test_x_num = test_x_num.fillna(value=0)

### string features 
test_x_str = test_data.iloc[:,[1,2,-4,-5,5]]
test_x_str = test_x_str.fillna(value='None')

train_x_str = train_data.iloc[:,[1,2,-4,-5,5]]
train_x_str = train_x_str.fillna(value='None')




In [39]:
#################### Processing ####################
feat_enc = OneHotEncoder()
feat_enc.fit(np.vstack([train_x_str, test_x_str]))
train_x_str_enc = feat_enc.transform(train_x_str).A
test_x_str_enc = feat_enc.transform(test_x_str).A

train_x_final=np.hstack((train_x_str_enc,train_x_num))
test_x_final=np.hstack((test_x_str_enc,test_x_num))

pca=PCA()
#pca.fit_transform(train_x_final)
#pca.fit_transform(test_x_final)

In [40]:
#################### Training ####################

classif = OneVsRestClassifier(RandomForestClassifier())


### Fit
print('... ... ... ... ... ... ... ... ... ... Training')
t0=time.time()
classif.fit(train_x_final, train_y)
print("training time:", round(time.time()-t0, 3), "s")

### Predict
print('... ... ... ... ... ... ... ... ... ... Predicting')
t1=time.time()
pred_y = classif.predict_proba(test_x_final)
print("prediction time:", round(time.time()-t1, 3), "s")
print('prediction labels shape:', pred_y.shape)

### Saving the submission file
pred_data = pd.DataFrame(pred_y, columns=['updates', 'personal', 'promotions','forums', 'purchases', 'travel','spam', 'social'])
pred_data.to_csv("randomforest.csv", index=True, index_label='Id')

... ... ... ... ... ... ... ... ... ... Training
training time: 405.222 s
... ... ... ... ... ... ... ... ... ... Predicting
prediction time: 9.141 s
prediction labels shape: (17002, 8)
